In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [2]:
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, losses
# GPUが利用可能か確認
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


Using device: cuda


In [3]:
# データの読み込み
train_data = pd.read_csv(r'/kaggle/input/llm-classification-finetuning/train.csv')
test_data = pd.read_csv(r'/kaggle/input/llm-classification-finetuning/test.csv')
labels = np.zeros(len(train_data), dtype=int)
labels[train_data['winner_model_a'] == 1] = 0  # Aの勝ち
labels[train_data['winner_model_b'] == 1] = 1  # Bの勝ち
labels[train_data['winner_tie'] == 1] = 2    # 同点


In [4]:
def extract_better_worse_responses(train_data):
    """
    トレーニングデータから好まれる回答と好まれない回答のペアを抽出する関数
    
    Args:
        train_data (pd.DataFrame): 元のトレーニングデータフレーム
        
    Returns:
        tuple: (prompts, better_responses, worse_responses)のタプル
    """
    prompts = []
    better_responses = []
    worse_responses = []
    
    for _, row in train_data.iterrows():
        prompt = row['prompt']
        
        # 好まれる回答と好まれない回答を特定
        if row['winner_model_a'] == 1:  # モデルAが勝者
            better_response = row['response_a']
            worse_response = row['response_b']
            prompts.append(prompt)
            better_responses.append(better_response)
            worse_responses.append(worse_response)
            
        elif row['winner_model_b'] == 1:  # モデルBが勝者
            better_response = row['response_b']
            worse_response = row['response_a']
            prompts.append(prompt)
            better_responses.append(better_response)
            worse_responses.append(worse_response)
            
        # 同点の場合はスキップ（または別の処理を追加することも可能）
        # 同点の場合も含める場合は以下のコメントを解除
        #elif row['winner_tie'] == 1:
            # 同点の場合の処理（例えば、両方とも「better」として扱うなど）
    
    return prompts, better_responses, worse_responses

# 関数を使用してデータを抽出
prompts, better_responses, worse_responses = extract_better_worse_responses(train_data)

print(f"抽出されたサンプル数: {len(prompts)}")
print(f"最初の例:\nプロンプト: {prompts[0][:100]}...\n好まれる回答: {better_responses[0][:100]}...\n好まれない回答: {worse_responses[0][:100]}...")

抽出されたサンプル数: 39716
最初の例:
プロンプト: ["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, ...
好まれる回答: ["The question of whether it is morally right to aim for a certain percentage of females in manageri...
好まれない回答: ["As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of ...


In [5]:
# Wandbをオフにするため、環境変数を設定
import os
os.environ["WANDB_DISABLED"] = "true"
# モデルのロード
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# 学習率の設定（通常はデフォルトより小さい値で開始）
learning_rate = 2e-5  # SBERTの微調整には2e-5〜5e-5が一般的

# ウェイトディケイの設定
weight_decay = 0.01  # 過学習防止のため

# マージン値の設定（トリプレットロスのパラメータ）
margin = 0.5  # 正例と負例の埋め込み間の距離差をどれだけ取るか

# トレーニングサンプルの作成
train_examples = []
for prompt, better, worse in zip(prompts, better_responses, worse_responses):
    train_examples.append(InputExample(texts=[prompt, better, worse]))

# バッチサイズの設定（大きすぎるとメモリ不足に、小さすぎると学習が安定しない）
batch_size = 16

# エポック数（多すぎると過学習するリスクあり）
num_epochs = 3

# データローダーの作成
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# トリプレットロスの設定（マージンを指定）
train_loss = losses.TripletLoss(model=model, triplet_margin=margin)

# 早期停止のコールバック設定
from sentence_transformers.evaluation import TripletEvaluator
# 検証用データセットを作成（全データの一部を使用）
dev_samples = train_examples[:len(train_examples)//10]  # 10%を検証に使用
evaluator = TripletEvaluator.from_input_examples(dev_samples)

# オプティマイザの設定
optimizer_params = {'lr': learning_rate, 'weight_decay': weight_decay}

# ファインチューニングの実行
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    evaluator=evaluator,
    evaluation_steps=1000,  # 1000バッチごとに評価
    warmup_steps=int(len(train_dataloader) * 0.1),  # 全体の10%をウォームアップに
    optimizer_params=optimizer_params,
    optimizer_class=torch.optim.AdamW,  # AdamWオプティマイザを使用
    scheduler='WarmupLinear',  # 学習率スケジューラ
    show_progress_bar=True,
    output_path='fine-tuned-sbert-model',
    save_best_model=True  # 最良のモデルを保存
)
# ファインチューニングされたモデルを使用してデータを再エンコード
fine_tuned_embedding_p = model.encode(train_data["prompt"])
fine_tuned_embedding_A = model.encode(train_data["response_a"])
fine_tuned_embedding_B = model.encode(train_data["response_b"])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss,Cosine Accuracy
1000,0.420900,No log,0.653740
1242,0.420900,No log,0.663813
2000,0.388600,No log,0.689751
2484,0.388600,No log,0.706371
3000,0.359300,No log,0.717955
3726,0.356400,No log,0.743893


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

In [6]:
np.save('fine_tuned_embedding_p.npy', fine_tuned_embedding_p)
np.save('fine_tuned_embedding_A.npy', fine_tuned_embedding_A)
np.save('fine_tuned_embedding_B.npy', fine_tuned_embedding_B)

In [7]:
class ParallelResponseEvaluator(nn.Module):
    """
    プロンプトを基準に回答を並列評価するモデル
    
    プロンプトを基準として、回答AとBをそれぞれ独立に評価し、
    その評価結果を比較して最終的な好まれやすさを予測します。
    """
    def __init__(self, input_dim=384, hidden_dim=256, num_heads=4, num_layers=2, dropout=0.1):
        """
        初期化関数
        
        Args:
            input_dim (int): 入力特徴量の次元数（SBERTの場合通常は384次元）
            hidden_dim (int): 隠れ層の次元数
            num_heads (int): 注意機構のヘッド数
            num_layers (int): Transformerエンコーダー層の数
            dropout (float): ドロップアウト率
        """
        super(ParallelResponseEvaluator, self).__init__()
        
        # 特徴量の次元
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # プロンプト処理用のTransformerエンコーダー
        self.prompt_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=input_dim,
                nhead=num_heads,
                dim_feedforward=hidden_dim,
                dropout=dropout,
                batch_first=True
            ),
            num_layers=num_layers
        )
        
        # クロスアテンション層（プロンプトと回答間の関係を処理）
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=input_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # 回答固有の特徴を抽出するための変換層
        self.response_feature_extractor = nn.Sequential(
            nn.Linear(input_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # 比較層（回答AとBの特徴を比較）
        self.comparison_layer = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # 出力層（3クラス分類）
        self.classifier = nn.Linear(hidden_dim // 2, 3)
        
        # 補助的な評価モジュール（各回答の個別スコア計算用）
        self.response_scorer = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1)
        )
        
        # モデルの重みを初期化
        self._init_weights()
        
    def _init_weights(self):
        """モデルの重みを適切に初期化する関数"""
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def process_response(self, prompt_encoding, response_encoding):
        """
        プロンプトと1つの回答の関係を処理する
        
        Args:
            prompt_encoding (Tensor): プロンプトのエンコーディング [batch_size, 1, input_dim]
            response_encoding (Tensor): 回答のエンコーディング [batch_size, 1, input_dim]
            
        Returns:
            Tensor: 回答の特徴表現 [batch_size, hidden_dim]
        """
        # クロスアテンション（プロンプトをクエリ、回答をキー・バリューとして）
        # 注: クロスアテンションでは、プロンプトがクエリとして機能し、回答の関連部分に注目します
        attn_output, _ = self.cross_attention(
            prompt_encoding,  # クエリ: プロンプト
            response_encoding,  # キー: 回答
            response_encoding   # バリュー: 回答
        )
        
        # 結合して豊かな特徴表現を作成
        # プロンプトに基づく注意の結果と元の回答の両方を使用
        combined = torch.cat([attn_output, response_encoding], dim=2)
        combined = combined.squeeze(1)  # [batch_size, 1, input_dim*2] -> [batch_size, input_dim*2]
        
        # 特徴抽出
        response_features = self.response_feature_extractor(combined)
        
        return response_features
        
    def forward(self, prompt_emb, response_a_emb, response_b_emb):
        """
        順伝播関数
        
        Args:
            prompt_emb (Tensor): プロンプトの埋め込み [batch_size, input_dim]
            response_a_emb (Tensor): 回答Aの埋め込み [batch_size, input_dim]
            response_b_emb (Tensor): 回答Bの埋め込み [batch_size, input_dim]
            
        Returns:
            Tensor: 3クラスの確率 [batch_size, 3]
        """
        batch_size = prompt_emb.size(0)
        
        # 次元を追加してTransformerに適した形に変換（シーケンス長1）
        prompt_emb = prompt_emb.unsqueeze(1)  # [batch_size, 1, input_dim]
        response_a_emb = response_a_emb.unsqueeze(1)  # [batch_size, 1, input_dim]
        response_b_emb = response_b_emb.unsqueeze(1)  # [batch_size, 1, input_dim]
        
        # プロンプトのエンコーディング
        prompt_encoding = self.prompt_encoder(prompt_emb)
        
        # 回答AとBを独立に処理
        response_a_features = self.process_response(prompt_encoding, response_a_emb)
        response_b_features = self.process_response(prompt_encoding, response_b_emb)
        
        # 個別のスコア計算（補助的な予測として）
        score_a = self.response_scorer(response_a_features)
        score_b = self.response_scorer(response_b_features)
        
        # 回答AとBの特徴を結合して比較
        comparison_features = torch.cat([response_a_features, response_b_features], dim=1)
        comparison_output = self.comparison_layer(comparison_features)
        
        # 最終的な分類（3クラス）
        logits = self.classifier(comparison_output)
        
        # スコア差を考慮した調整（オプション）
        score_diff = score_a - score_b
        # ここではスコア差を直接利用しないが、モデルの拡張として使用可能
        
        return logits, score_a, score_b

In [8]:
class DualEncoderModel(nn.Module):
    """
    デュアルエンコーダーアーキテクチャを使用した好まれやすさ予測モデル
    
    プロンプトと回答のペアをそれぞれ評価し、その結果を比較します。
    回答AとBは順序によらず同等に扱われます。
    """
    def __init__(self, input_dim=384, hidden_dim=256, dropout=0.1):
        """
        初期化関数
        
        Args:
            input_dim (int): 入力特徴量の次元数（SBERTの場合通常は384次元）
            hidden_dim (int): 隠れ層の次元数
            dropout (float): ドロップアウト率
        """
        super(DualEncoderModel, self).__init__()
        
        # プロンプト-回答のマッチングを評価するエンコーダー
        self.pair_encoder = nn.Sequential(
            nn.Linear(input_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # ペアの特徴から個別スコアを計算
        self.pair_scorer = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1)
        )
        
        # 好まれやすさの差から最終的な分類を行う層
        self.preference_classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 2, hidden_dim),  # 2つのペア特徴+スコア差
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 3)  # 3クラス分類
        )
        
        # モデルの重みを初期化
        self._init_weights()
        
    def _init_weights(self):
        """モデルの重みを適切に初期化する関数"""
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        
    def encode_pair(self, prompt_emb, response_emb):
        """
        プロンプトと回答のペアをエンコードする
        
        Args:
            prompt_emb (Tensor): プロンプトの埋め込み [batch_size, input_dim]
            response_emb (Tensor): 回答の埋め込み [batch_size, input_dim]
            
        Returns:
            tuple: (ペアの特徴, 適合度スコア)
        """
        # プロンプトと回答を結合
        pair_input = torch.cat([prompt_emb, response_emb], dim=1)
        
        # ペアの特徴を抽出
        pair_features = self.pair_encoder(pair_input)
        
        # 適合度スコアを計算
        score = self.pair_scorer(pair_features)
        
        return pair_features, score
        
    def forward(self, prompt_emb, response_a_emb, response_b_emb):
        """
        順伝播関数
        
        Args:
            prompt_emb (Tensor): プロンプトの埋め込み [batch_size, input_dim]
            response_a_emb (Tensor): 回答Aの埋め込み [batch_size, input_dim]
            response_b_emb (Tensor): 回答Bの埋め込み [batch_size, input_dim]
            
        Returns:
            Tensor: 3クラスの確率 [batch_size, 3]
        """
        # プロンプト-回答Aのペアを評価
        pair_a_features, score_a = self.encode_pair(prompt_emb, response_a_emb)
        
        # プロンプト-回答Bのペアを評価
        pair_b_features, score_b = self.encode_pair(prompt_emb, response_b_emb)
        
        # スコアの差を計算
        score_diff = score_a - score_b
        score_abs_diff = torch.abs(score_diff)
        
        # すべての特徴を結合
        combined_features = torch.cat([
            pair_a_features, 
            pair_b_features, 
            score_diff,  # 方向性のある差（A-B）
            score_abs_diff  # 絶対的な差（同等かどうかの判断に役立つ）
        ], dim=1)
        
        # 最終的な分類
        logits = self.preference_classifier(combined_features)
        
        return logits, score_a, score_b


In [9]:
# データセットクラス（より明示的な並列構造用）
class ParallelResponseDataset(Dataset):
    """
    回答を並列に扱うためのデータセットクラス
    """
    def __init__(self, prompt_embeddings, response_a_embeddings, response_b_embeddings, labels=None):
        """
        初期化関数
        
        Args:
            prompt_embeddings (numpy.ndarray): プロンプトの埋め込み [N, embed_dim]
            response_a_embeddings (numpy.ndarray): 回答Aの埋め込み [N, embed_dim]
            response_b_embeddings (numpy.ndarray): 回答Bの埋め込み [N, embed_dim]
            labels (numpy.ndarray, optional): ラベル [N]
                                             0: Aが好まれる, 1: Bが好まれる, 2: 同等
        """
        self.prompt_embeddings = torch.tensor(prompt_embeddings, dtype=torch.float32)
        self.response_a_embeddings = torch.tensor(response_a_embeddings, dtype=torch.float32)
        self.response_b_embeddings = torch.tensor(response_b_embeddings, dtype=torch.float32)
        
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.long)
        else:
            self.labels = None
            
        self.has_labels = labels is not None
        
    def __len__(self):
        """データセットの長さを返す"""
        return len(self.prompt_embeddings)
    
    def __getitem__(self, idx):
        """インデックスに対応するデータを返す"""
        if self.has_labels:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx],
                self.labels[idx]
            )
        else:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx]
            )



In [10]:

# 訓練関数
def train_parallel_model(model, train_loader, val_loader, n_epochs=20, lr=0.001, 
                         device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    並列処理モデルを訓練する関数
    
    Args:
        model: 訓練するモデル
        train_loader: 訓練データローダー
        val_loader: 検証データローダー
        n_epochs: エポック数
        lr: 学習率
        device: 使用デバイス ('cuda' or 'cpu')
        
    Returns:
        dict: 訓練結果（ベストモデル、履歴など）
    """
    # モデルをデバイスに移動
    model = model.to(device)
    print(f"Using device: {device}")
    
    # 損失関数と最適化アルゴリズム
    criterion = nn.CrossEntropyLoss()
    aux_criterion = nn.MSELoss()  # 補助的なスコア予測用
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # 学習率スケジューラ
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    
    # 結果記録用の変数
    best_val_acc = 0.0
    best_model_state = None
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rates': []
    }
    
    # エポックごとの訓練ループ
    for epoch in range(n_epochs):
        # 現在の学習率を記録
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)
        
        print(f"\nEpoch {epoch+1}/{n_epochs}, LR: {current_lr:.6f}")
        
        # 訓練フェーズ
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_data in train_loader:
            prompt_emb, resp_a_emb, resp_b_emb, labels = [b.to(device) for b in batch_data]
            
            # 勾配をゼロに初期化
            optimizer.zero_grad()
            
            # 順伝播
            if isinstance(model, ParallelResponseEvaluator) or isinstance(model, DualEncoderModel):
                outputs, score_a, score_b = model(prompt_emb, resp_a_emb, resp_b_emb)
                
                # メイン損失（分類）
                main_loss = criterion(outputs, labels)
                
                # 補助的な損失（スコア予測）
                # ラベルに基づいて期待されるスコアの差を設定
                # 0: Aが好まれる -> スコア差を正に
                # 1: Bが好まれる -> スコア差を負に
                # 2: 同等 -> スコア差を0に
                expected_scores = torch.zeros_like(score_a)
                expected_scores[labels == 0] = 1.0  # Aが好まれる
                expected_scores[labels == 1] = -1.0  # Bが好まれる
                # 同等の場合は0のまま
                
                aux_loss = aux_criterion(score_a - score_b, expected_scores)
                
                # 総合損失
                loss = main_loss + 0.5 * aux_loss  # 補助損失の重みは調整可能
            else:
                outputs = model(prompt_emb, resp_a_emb, resp_b_emb)
                loss = criterion(outputs, labels)
            
            # 逆伝播と最適化
            loss.backward()
            optimizer.step()
            
            # 統計の更新
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # 訓練統計の計算
        train_loss /= len(train_loader)
        train_acc = train_correct / train_total
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        
        # 検証フェーズ
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for batch_data in val_loader:
                prompt_emb, resp_a_emb, resp_b_emb, labels = [b.to(device) for b in batch_data]
                
                # 順伝播
                if isinstance(model, ParallelResponseEvaluator) or isinstance(model, DualEncoderModel):
                    outputs, _, _ = model(prompt_emb, resp_a_emb, resp_b_emb)
                else:
                    outputs = model(prompt_emb, resp_a_emb, resp_b_emb)
                
                # 損失の計算
                loss = criterion(outputs, labels)
                
                # 統計の更新
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                
                # 予測とターゲットを記録
                val_preds.extend(predicted.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())
        
        # 検証統計の計算
        val_loss /= len(val_loader)
        val_preds = np.array(val_preds)
        val_targets = np.array(val_targets)
        val_acc = accuracy_score(val_targets, val_preds)
        
        # 混同行列を計算
        conf_matrix = confusion_matrix(val_targets, val_preds)
        
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print("Confusion Matrix:")
        print(conf_matrix)
        
        # 学習率スケジューラを更新
        scheduler.step(val_acc)
        
        # ベストモデルの保存
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            print(f"New best model with validation accuracy: {val_acc:.4f}")
    
    # 訓練終了後、ベストモデルを復元
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        
    return {
        'model': model,
        'best_val_acc': best_val_acc,
        'history': history
    }

In [11]:
prompt_embeddings = fine_tuned_embedding_p
response_a_embeddings = fine_tuned_embedding_A
response_b_embeddings = fine_tuned_embedding_B

num_samples = prompt_embeddings.shape[0]
embed_dim = prompt_embeddings.shape[1]

print(num_samples)
print(embed_dim)


# ラベルがGPU上にある場合
if isinstance(labels, torch.Tensor) and labels.is_cuda:
    labels_cpu = labels.cpu().numpy()
else:
    labels_cpu = labels

train_idx, test_idx = train_test_split(
    np.arange(num_samples), test_size=0.2, random_state=42, stratify=labels_cpu
)

train_idx, val_idx = train_test_split(
        train_idx, test_size=0.25, random_state=42, stratify=labels[train_idx]
)
    

# インデックスに基づいてデータを分割
p_train, r_a_train, r_b_train = prompt_embeddings[train_idx], response_a_embeddings[train_idx], response_b_embeddings[train_idx]
p_val, r_a_val, r_b_val = prompt_embeddings[val_idx], response_a_embeddings[val_idx], response_b_embeddings[val_idx]
p_test, r_a_test, r_b_test = prompt_embeddings[test_idx], response_a_embeddings[test_idx], response_b_embeddings[test_idx]

y_train, y_val, y_test = labels[train_idx], labels[val_idx], labels[test_idx]

# ========== データローダーの作成 ==========
# データセットの作成
train_dataset = ParallelResponseDataset(p_train, r_a_train, r_b_train, y_train)
val_dataset = ParallelResponseDataset(p_val, r_a_val, r_b_val, y_val)
test_dataset = ParallelResponseDataset(p_test, r_a_test, r_b_test, y_test)

# データローダーの作成
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


# ========== モデルの作成と訓練 ==========
# どちらのモデルを使用するか選択
model_type = "parallel_evaluator"  # または "parallel_evaluator"

if model_type == "parallel_evaluator":
    model = ParallelResponseEvaluator(
        input_dim=embed_dim,      # 入力の次元数
        hidden_dim=256,           # 隠れ層の次元数
        num_heads=4,              # 注意ヘッドの数
        num_layers=2,             # エンコーダー層の数
        dropout=0.1               # ドロップアウト率
    )
else:  # dual_encoder
    model = DualEncoderModel(
        input_dim=embed_dim,      # 入力の次元数
        hidden_dim=256,           # 隠れ層の次元数
        dropout=0.1               # ドロップアウト率
    )

# モデルの要約を表示
print(model)

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルの訓練
training_results = train_parallel_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=20,          # エポック数
    lr=0.001,             # 初期学習率
    device=device         # 使用デバイス
)

# ========== モデルの評価 ==========
# テストデータでの評価
model.eval()
test_preds = []
test_targets = []

with torch.no_grad():
    for batch_data in test_loader:
        prompt_emb, resp_a_emb, resp_b_emb, labels = [b.to(device) for b in batch_data]
        
        # 順伝播
        if isinstance(model, ParallelResponseEvaluator) or isinstance(model, DualEncoderModel):
            outputs, _, _ = model(prompt_emb, resp_a_emb, resp_b_emb)
        else:
            outputs = model(prompt_emb, resp_a_emb, resp_b_emb)
            
        _, predicted = torch.max(outputs, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_targets.extend(labels.cpu().numpy())

# 精度の計算
test_acc = accuracy_score(test_targets, test_preds)
print(f"Test Accuracy: {test_acc:.4f}")

# 混同行列の表示
conf_matrix = confusion_matrix(test_targets, test_preds)
print("Confusion Matrix:")
print(conf_matrix)


57477
384
ParallelResponseEvaluator(
  (prompt_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
        (linear1): Linear(in_features=384, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=384, bias=True)
        (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (cross_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
  )
  (response_feature_extractor): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1